In [26]:
!pip install tensorflow opencv-python

In [29]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define paths for Ref and Non-Ref images
base_dir = '/content/drive/My Drive/dataset_Ref_Non_Ref'
reflection_dir = os.path.join(base_dir, 'Ref')  # Ref images
clear_dir = os.path.join(base_dir, 'Non_Ref')  # Non-Ref images

def load_images_from_folder(folder, target_size=(256, 256)):
    images = []
    for filename in sorted(os.listdir(folder)):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, target_size)
            img = img / 255.0
            images.append(img)
    return np.array(images)


X_train = load_images_from_folder(reflection_dir)  # Ref images
Y_train = load_images_from_folder(clear_dir)  # Non-Ref images

print(f"Loaded {len(X_train)} images with reflections.")
print(f"Loaded {len(Y_train)} clear images.")

assert len(X_train) == len(Y_train), "Mismatch between input and output datasets."

def unet_model(input_shape=(256, 256, 3)):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # Decoder
    u5 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(3, (1, 1), activation='sigmoid')(c7)

    model = models.Model(inputs, outputs)
    return model

# Compile  model
model = unet_model()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

batch_size = 16
num_epochs = 50
steps_per_epoch = len(X_train) // batch_size

def data_generator(X, Y, batch_size):
    while True:
        for i in range(0, len(X), batch_size):
            X_batch = X[i:i+batch_size]
            Y_batch = Y[i:i+batch_size]
            yield (np.array(X_batch), np.array(Y_batch))

train_gen = data_generator(X_train, Y_train, batch_size)

history = model.fit(train_gen, epochs=num_epochs, steps_per_epoch=steps_per_epoch)

model.save('reflection_removal_unet.h5')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 30 images with reflections.
Loaded 30 clear images.


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8             │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_120 (Conv2D)       │ (None, 256, 256, 64)   │          1,792 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_121 (Conv2D)       │ (None, 256, 256, 64)   │         36,928 │ conv2d_120[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_24          │ (None, 128, 128, 64)   │              0 │ conv2d_121[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_122 (Conv2D)       │ (None, 128, 128, 128)  │         73,856 │ max_pooling2d_24[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_123 (Conv2D)       │ (None, 128, 128, 128)  │        147,584 │ conv2d_122[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_25          │ (None, 64, 64, 128)    │              0 │ conv2d_123[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_124 (Conv2D)       │ (None, 64, 64, 256)    │        295,168 │ max_pooling2d_25[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_125 (Conv2D)       │ (None, 64, 64, 256)    │        590,080 │ conv2d_124[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_26          │ (None, 32, 32, 256)    │              0 │ conv2d_125[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_126 (Conv2D)       │ (None, 32, 32, 512)    │      1,180,160 │ max_pooling2d_26[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_127 (Conv2D)       │ (None, 32, 32, 512)    │      2,359,808 │ conv2d_126[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_24       │ (None, 64, 64, 256)    │        524,544 │ conv2d_127[0][0]       │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_24            │ (None, 64, 64, 512)    │              0 │ conv2d_transpose_24[0… │
│ (Concatenate)             │                        │                │ conv2d_125[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_128 (Conv2D)       │ (None, 64, 64, 256)    │      1,179,904 │ concatenate_24[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_129 (Conv2D)       │ (None, 64, 64, 256)    │        590,080 │ conv2d_128[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_25  

 Total params: 7,697,475 (29.36 MB)

 Trainable params: 7,697,475 (29.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.8193 - loss: 0.0680
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.8025 - loss: 0.0322
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 612ms/step - accuracy: 0.8194 - loss: 0.0748
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 553ms/step - accuracy: 0.8025 - loss: 0.0316
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 612ms/step - accuracy: 0.8194 - loss: 0.0650
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 551ms/step - accuracy: 0.8025 - loss: 0.0332
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 601ms/step - accuracy: 0.8194 - loss: 0.0648
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 561ms/step - accuracy: 0.8025 - loss: 0.0338
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 602ms/step - accuracy: 0.8194 - loss: 0.0648
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 557ms/step - accuracy: 0.8025 - loss: 0.0331
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 612ms/step - accuracy: 0.8194 - loss: 0.0647
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 564ms/step - accuracy: 0.8025 - loss: 0.

In [30]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2
import os
import math
from google.colab import drive

# Mount Google
drive.mount('/content/drive')

base_dir = '/content/drive/My Drive/dataset_Ref_Non_Ref'
reflection_dir = os.path.join(base_dir, 'Ref')  # Ref images
clear_dir = os.path.join(base_dir, 'Non_Ref')  # Non-Ref images

def load_images_from_folder(folder, target_size=(256, 256)):
    images = []
    for filename in sorted(os.listdir(folder)):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, target_size)
            img = img / 255.0
            images.append(img)
    return np.array(images)



X_train = load_images_from_folder(reflection_dir)  # Ref images
Y_train = load_images_from_folder(clear_dir)  # Non-Ref images

print(f"Loaded {len(X_train)} images with reflections.")
print(f"Loaded {len(Y_train)} clear images.")

assert len(X_train) == len(Y_train), "Mismatch between input and output datasets."

def unet_model(input_shape=(256, 256, 3)):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # Decoder
    u5 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(3, (1, 1), activation='sigmoid')(c7)

    model = models.Model(inputs, outputs)
    return model

model = unet_model()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

split_idx = int(0.8 * len(X_train))
X_train, X_val = X_train[:split_idx], X_train[split_idx:]
Y_train, Y_val = Y_train[:split_idx], Y_train[split_idx:]

batch_size = 16
num_epochs = 50

# Calculate steps
steps_per_epoch = math.ceil(len(X_train) / batch_size)
validation_steps = math.ceil(len(X_val) / batch_size)

# Custom data generator
def data_generator(X, Y, batch_size):
    while True:
        indices = np.random.permutation(len(X))
        for i in range(0, len(X), batch_size):
            batch_indices = indices[i:i + batch_size]
            X_batch = [X[j] for j in batch_indices]
            Y_batch = [Y[j] for j in batch_indices]
            yield np.array(X_batch), np.array(Y_batch)

train_gen = data_generator(X_train, Y_train, batch_size)
val_gen = data_generator(X_val, Y_val, batch_size)

# Train the model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=num_epochs,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)
model.save('reflection_removal_unet.h5')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 30 images with reflections.
Loaded 30 clear images.


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9             │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_135 (Conv2D)       │ (None, 256, 256, 64)   │          1,792 │ input_layer_9[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_136 (Conv2D)       │ (None, 256, 256, 64)   │         36,928 │ conv2d_135[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_27          │ (None, 128, 128, 64)   │              0 │ conv2d_136[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_137 (Conv2D)       │ (None, 128, 128, 128)  │         73,856 │ max_pooling2d_27[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_138 (Conv2D)       │ (None, 128, 128, 128)  │        147,584 │ conv2d_137[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_28          │ (None, 64, 64, 128)    │              0 │ conv2d_138[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_139 (Conv2D)       │ (None, 64, 64, 256)    │        295,168 │ max_pooling2d_28[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_140 (Conv2D)       │ (None, 64, 64, 256)    │        590,080 │ conv2d_139[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_29          │ (None, 32, 32, 256)    │              0 │ conv2d_140[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_141 (Conv2D)       │ (None, 32, 32, 512)    │      1,180,160 │ max_pooling2d_29[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_142 (Conv2D)       │ (None, 32, 32, 512)    │      2,359,808 │ conv2d_141[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_27       │ (None, 64, 64, 256)    │        524,544 │ conv2d_142[0][0]       │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_27            │ (None, 64, 64, 512)    │              0 │ conv2d_transpose_27[0… │
│ (Concatenate)             │                        │                │ conv2d_140[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_143 (Conv2D)       │ (None, 64, 64, 256)    │      1,179,904 │ concatenate_27[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_144 (Conv2D)       │ (None, 64, 64, 256)    │        590,080 │ conv2d_143[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_28  

 Total params: 7,697,475 (29.36 MB)

 Trainable params: 7,697,475 (29.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - accuracy: 0.8145 - loss: 0.0576 - val_accuracy: 0.7983 - val_loss: 0.0309
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 452ms/step - accuracy: 0.8145 - loss: 0.0567 - val_accuracy: 0.7983 - val_loss: 0.0314
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 465ms/step - accuracy: 0.8154 - loss: 0.0551 - val_accuracy: 0.7983 - val_loss: 0.0320
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 434ms/step - accuracy: 0.8149 - loss: 0.0530 - val_accuracy: 0.7983 - val_loss: 0.0317
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 431ms/step - accuracy: 0.8142 - loss: 0.0543 - val_accuracy: 0.7983 - val_loss: 0.0313
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 430ms/step - accuracy: 0.8141 - loss: 0.0529 - val_accuracy: 0.7983 - val_loss: 0.0306
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 442ms/step - accuracy: 0.8149 - loss: 0.0545 - val_accuracy: 0.7983 - val_loss: 0.0301
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 439ms/step - accuracy: 0.8137 - loss: 0.0525 - val_accuracy: 0.7983 - val_loss: 0

In [33]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
import math
from google.colab import drive

drive.mount('/content/drive')

# Define paths for Ref and Non-Ref images
base_dir = '/content/drive/My Drive/dataset_Ref_Non_Ref'
reflection_dir = os.path.join(base_dir, 'Ref')  # Ref images
clear_dir = os.path.join(base_dir, 'Non_Ref')  # Non-Ref images

def load_images_from_folder(folder, target_size=(256, 256)):
    images = []
    for filename in sorted(os.listdir(folder)):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, target_size)
            img = img / 255.0
            images.append(img)
    return np.array(images)


X_train = load_images_from_folder(reflection_dir)  # Ref images
Y_train = load_images_from_folder(clear_dir)  # Non-Ref images

print(f"Loaded {len(X_train)} images with reflections.")
print(f"Loaded {len(Y_train)} clear images.")

assert len(X_train) == len(Y_train), "Mismatch between input and output datasets."

# Define UNet model
def unet_model(input_shape=(256, 256, 3)):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # Decoder
    u5 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(3, (1, 1), activation='sigmoid')(c7)

    model = models.Model(inputs, outputs)
    return model

model = unet_model()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

split_idx = int(0.8 * len(X_train))
X_train, X_val = X_train[:split_idx], X_train[split_idx:]
Y_train, Y_val = Y_train[:split_idx], Y_train[split_idx:]

batch_size = 16
num_epochs = 50

steps_per_epoch = math.ceil(len(X_train) / batch_size)
validation_steps = math.ceil(len(X_val) / batch_size)

def data_generator(X, Y, batch_size):
    while True:
        indices = np.random.permutation(len(X))
        for i in range(0, len(X), batch_size):
            batch_indices = indices[i:i + batch_size]
            X_batch = [X[j] for j in batch_indices]
            Y_batch = [Y[j] for j in batch_indices]
            yield np.array(X_batch), np.array(Y_batch)

train_gen = data_generator(X_train, Y_train, batch_size)
val_gen = data_generator(X_val, Y_val, batch_size)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=num_epochs,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)
model.save('reflection_removal_unet.h5')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 30 images with reflections.
Loaded 30 clear images.


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12            │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_180 (Conv2D)       │ (None, 256, 256, 64)   │          1,792 │ input_layer_12[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_181 (Conv2D)       │ (None, 256, 256, 64)   │         36,928 │ conv2d_180[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_36          │ (None, 128, 128, 64)   │              0 │ conv2d_181[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_182 (Conv2D)       │ (None, 128, 128, 128)  │         73,856 │ max_pooling2d_36[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_183 (Conv2D)       │ (None, 128, 128, 128)  │        147,584 │ conv2d_182[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_37          │ (None, 64, 64, 128)    │              0 │ conv2d_183[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_184 (Conv2D)       │ (None, 64, 64, 256)    │        295,168 │ max_pooling2d_37[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_185 (Conv2D)       │ (None, 64, 64, 256)    │        590,080 │ conv2d_184[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_38          │ (None, 32, 32, 256)    │              0 │ conv2d_185[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_186 (Conv2D)       │ (None, 32, 32, 512)    │      1,180,160 │ max_pooling2d_38[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_187 (Conv2D)       │ (None, 32, 32, 512)    │      2,359,808 │ conv2d_186[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_36       │ (None, 64, 64, 256)    │        524,544 │ conv2d_187[0][0]       │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_36            │ (None, 64, 64, 512)    │              0 │ conv2d_transpose_36[0… │
│ (Concatenate)             │                        │                │ conv2d_185[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_188 (Conv2D)       │ (None, 64, 64, 256)    │      1,179,904 │ concatenate_36[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_189 (Conv2D)       │ (None, 64, 64, 256)    │        590,080 │ conv2d_188[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_37  

 Total params: 7,697,475 (29.36 MB)

 Trainable params: 7,697,475 (29.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 16s 4s/step - accuracy: 0.3242 - loss: 0.0585 - val_accuracy: 0.7968 - val_loss: 0.0313
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 429ms/step - accuracy: 0.8127 - loss: 0.0539 - val_accuracy: 0.7983 - val_loss: 0.0306
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 424ms/step - accuracy: 0.8146 - loss: 0.0526 - val_accuracy: 0.7983 - val_loss: 0.0305
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 419ms/step - accuracy: 0.8149 - loss: 0.0536 - val_accuracy: 0.7983 - val_loss: 0.0307
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 423ms/step - accuracy: 0.8145 - loss: 0.0561 - val_accuracy: 0.7983 - val_loss: 0.0308
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 431ms/step - accuracy: 0.8146 - loss: 0.0563 - val_accuracy: 0.7983 - val_loss: 0.0306
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 424ms/step - accuracy: 0.8141 - loss: 0.0545 - val_accuracy: 0.7983 - val_loss: 0.0302
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 431ms/step - accuracy: 0.8145 - loss: 0.0544 - val_accuracy: 0.7983 - val_loss: 0

In [35]:
from google.colab import drive
import shutil
import os

drive.mount('/content/drive', force_remount=True)

colab_model_path = '/content/improved_unet_reflection_removal.h5'

drive_model_folder = '/content/drive/MyDrive/your_model_folder1/'

os.makedirs(drive_model_folder, exist_ok=True)

drive_model_path = os.path.join(drive_model_folder, 'improved_unet_reflection_removal.h5')

if not os.path.exists(colab_model_path):
    print(f"Error: The model file does not exist at {colab_model_path}")
else:
    shutil.copy(colab_model_path, drive_model_path)

    if os.path.exists(drive_model_path):
        print(f"Model successfully saved to Google Drive at: {drive_model_path}")
    else:
        print("Error: Model not found in the specified Google Drive path.")


Mounted at /content/drive
Model successfully saved to Google Drive at: /content/drive/MyDrive/your_model_folder1/improved_unet_reflection_removal.h5
